# Add batch effects

Say we are interested in identifying genes that differentiate between disease vs normal states.  However our dataset includes samples from different tissues or time points and there are variations in gene expression that are due to these other conditions and do not have to do with disease state.  These non-relevant variations in the data are called *batch effects*.  

We want to model these batch effects.  To do this we will:
1. Partition our simulated data into n batches
2. For each partition we will randomly shift the expression data.  We randomly generate a binary vector of length=number of genes (*offset vector*).  This vector will serve as the direction that we will shift to.  Then we also have a random scalar that will tell us how big of a step to take in our random direction (*stretch factor*).  We shift our partitioned data by: batch effect partition = partitioned data + stretch factor * offset vector
3. Repeat this for each partition
4. Append all batch effect partitions together


In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import glob
import umap
import pickle
import seaborn as sns
import warnings
warnings.filterwarnings(action='once')

from ggplot import *
from sklearn.decomposition import PCA
from numpy.random import seed
randomState = 123
seed(randomState)

/home/alexandra/anaconda3/envs/batch_effects/lib/python3.5/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/alexandra/anaconda3/envs/batch_effects/lib/python3.5/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/alexandra/anaconda3/envs/batch_effects/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Parameters
analysis_name = 'experiment_0'
NN_architecture = 'NN_2500_30'
num_PCs = 100
num_simulations = 10
num_batches = [1,2,5,10,20,50,100,500,1000,2000,3000,6000]
#num_batches = [1,2,3,4,5,6,7,8,9,10,15,20,50,100,500,800,1000,2000,3000,4000,5000,6000]

In [3]:
# Create directories
base_dir = os.path.abspath(os.path.join(os.getcwd(),"../.."))

new_dir = os.path.join(
    base_dir,
    "data",
    "batch_simulated")

analysis_dir = os.path.join(new_dir, analysis_name)

if os.path.exists(analysis_dir):
    print('directory already exists: {}'.format(analysis_dir))
else:
    print('creating new directory: {}'.format(analysis_dir))
os.makedirs(analysis_dir, exist_ok=True)

directory already exists: /home/alexandra/Documents/Repos/Batch_effects_simulation/data/batch_simulated/experiment_0


In [4]:
# Load arguments
simulated_data_file = os.path.join(
    base_dir,
    "data",
    "simulated",
    analysis_name,
    "simulated_data.txt")

umap_model_file = umap_model_file = os.path.join(
    base_dir,
    "models",  
    NN_architecture,
    "umap_model.pkl")

In [5]:
# Read in UMAP model
infile = open(umap_model_file, 'rb')
umap_model = pickle.load(infile)
infile.close()

In [6]:
# Read in data
simulated_data = pd.read_table(
    simulated_data_file,
    header=0, 
    index_col=0,
    sep='\t')

simulated_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,5539,5540,5541,5542,5543,5544,5545,5546,5547,5548
0,0.690384,0.642501,0.454786,0.650650,0.374857,0.414020,0.353704,0.566726,0.447497,0.165201,...,0.375559,0.600063,0.562897,0.640035,0.661894,0.325466,0.576836,0.567330,0.708088,0.615353
1,0.691060,0.655274,0.527149,0.680636,0.371842,0.443242,0.374220,0.533293,0.502785,0.165815,...,0.290522,0.591411,0.609883,0.585498,0.596831,0.162511,0.459705,0.531669,0.710235,0.698571
2,0.826005,0.694632,0.510346,0.642764,0.611429,0.517733,0.344205,0.631357,0.663116,0.199852,...,0.589395,0.581802,0.596350,0.671666,0.753091,0.160510,0.521740,0.511157,0.747738,0.728167
3,0.600721,0.564944,0.417176,0.594936,0.382868,0.440063,0.387032,0.466111,0.402363,0.223858,...,0.345520,0.547836,0.470917,0.460431,0.564647,0.256127,0.509963,0.348220,0.579351,0.583487
4,0.621544,0.615939,0.473489,0.599652,0.401605,0.481008,0.364476,0.444714,0.447605,0.215759,...,0.450535,0.532127,0.588728,0.547413,0.578212,0.234255,0.424777,0.501985,0.704726,0.717408
5,0.735929,0.732768,0.545550,0.632979,0.460602,0.437013,0.325476,0.572898,0.682289,0.203184,...,0.414369,0.576000,0.692773,0.643043,0.746655,0.215016,0.509911,0.638528,0.775373,0.746010
6,0.643262,0.647871,0.449706,0.552101,0.407846,0.421758,0.371682,0.551611,0.530684,0.185147,...,0.392800,0.503936,0.558234,0.505509,0.600357,0.213806,0.478426,0.443200,0.641380,0.686138
7,0.517200,0.567826,0.363922,0.483920,0.346046,0.440530,0.442479,0.474701,0.372429,0.156235,...,0.368144,0.488229,0.461496,0.446372,0.480050,0.364032,0.477898,0.388174,0.528046,0.492165
8,0.746649,0.745975,0.622174,0.748726,0.455695,0.499827,0.260720,0.625723,0.640932,0.206239,...,0.421102,0.636195,0.681262,0.676396,0.637278,0.241293,0.594934,0.663525,0.725401,0.708356
9,0.580710,0.586180,0.419463,0.592984,0.368153,0.405530,0.403313,0.445573,0.449738,0.212873,...,0.419540,0.534721,0.509361,0.491707,0.524512,0.257988,0.408664,0.429760,0.594038,0.572804


In [7]:
# Add batch effects
# ADD MULTIPLE SIMULATION RUNS
num_simulated_samples = simulated_data.shape[0]
num_genes = simulated_data.shape[1]
subset_genes_to_change = np.random.RandomState(randomState).choice([0, 1], size=(num_genes), p=[3./4, 1./4])
    
for i in num_batches:
    print('Creating simulated data with {} batches..'.format(i))
    
    batch_file = os.path.join(
            base_dir,
            "data",
            "batch_simulated",
            analysis_name,
            "Batch_"+str(i)+".txt")
    
    num_samples_per_batch = int(num_simulated_samples/i)
    
    if i == 1:        
        simulated_data.to_csv(batch_file, sep='\t')
        
    else:  
        batch_data_df = pd.DataFrame()
        for j in range(i):
            
            stretch_factor = np.random.uniform(1.0,1.5)
            
            # Randomly select samples
            batch_df = simulated_data.sample(n=num_samples_per_batch, frac=None, replace=False)
            batch_df.columns = batch_df.columns.astype(str)

            # Add batch effect
            subset_genes_to_change_tile = pd.DataFrame(
                pd.np.tile(
                    subset_genes_to_change,
                    (num_samples_per_batch, 1)),
                index=batch_df.index)

            offset_vector = pd.DataFrame(subset_genes_to_change_tile*stretch_factor)
            offset_vector.columns = offset_vector.columns.astype(str)
            batch_df = batch_df + offset_vector
            
            #batch_df = batch_df*stretch_factor

            # if any exceed 1 then set to 1 since gene expression is normalized
            batch_df[batch_df>=1.0] = 1.0


            # Append batched together
            batch_data_df = batch_data_df.append(batch_df)

            # Select a new direction (i.e. a new subset of genes to change)
            np.random.shuffle(subset_genes_to_change)
            
        # Save
        batch_data_df.to_csv(batch_file, sep='\t')

Creating simulated data with 1 batches..
Creating simulated data with 2 batches..
Creating simulated data with 5 batches..
Creating simulated data with 10 batches..
Creating simulated data with 20 batches..
Creating simulated data with 50 batches..
Creating simulated data with 100 batches..
Creating simulated data with 500 batches..
Creating simulated data with 1000 batches..
Creating simulated data with 2000 batches..
Creating simulated data with 3000 batches..
Creating simulated data with 6000 batches..


## Plot batch data using UMAP

In [8]:
"""
# Plot generated data 

for i in num_batches:
    batch_data_file = os.path.join(
        base_dir,
        "data",
        "batch_simulated",
        analysis_name,
        "Batch_"+str(i)+".txt")
    
    batch_data = pd.read_table(
        batch_data_file,
        header=0,
        sep='\t',
        index_col=0)
    
    # UMAP embedding of decoded batch data
    batch_data_UMAPencoded = umap_model.transform(batch_data)
    batch_data_UMAPencoded_df = pd.DataFrame(data=batch_data_UMAPencoded,
                                             index=batch_data.index,
                                             columns=['1','2'])
    
        
    g = ggplot(aes(x='1',y='2'), data=batch_data_UMAPencoded_df) + \
                geom_point(alpha=0.5) + \
                scale_color_brewer(type='qual', palette='Set2') + \
                ggtitle("{} Batches".format(i))
    
    print(g)"""

'\n# Plot generated data \n\nfor i in num_batches:\n    batch_data_file = os.path.join(\n        base_dir,\n        "data",\n        "batch_simulated",\n        analysis_name,\n        "Batch_"+str(i)+".txt")\n    \n    batch_data = pd.read_table(\n        batch_data_file,\n        header=0,\n        sep=\'\t\',\n        index_col=0)\n    \n    # UMAP embedding of decoded batch data\n    batch_data_UMAPencoded = umap_model.transform(batch_data)\n    batch_data_UMAPencoded_df = pd.DataFrame(data=batch_data_UMAPencoded,\n                                             index=batch_data.index,\n                                             columns=[\'1\',\'2\'])\n    \n        \n    g = ggplot(aes(x=\'1\',y=\'2\'), data=batch_data_UMAPencoded_df) +                 geom_point(alpha=0.5) +                 scale_color_brewer(type=\'qual\', palette=\'Set2\') +                 ggtitle("{} Batches".format(i))\n    \n    print(g)'

## Plot batch data using PCA

In [9]:
# Plot generated data 

for i in num_batches:
    batch_data_file = os.path.join(
        base_dir,
        "data",
        "batch_simulated",
        analysis_name,
        "Batch_"+str(i)+".txt")
    
    batch_data = pd.read_table(
        batch_data_file,
        header=0,
        sep='\t',
        index_col=0)
    
    # PCA projection    
    pca = PCA(n_components=num_PCs)
    batch_data_PCAencoded = pca.fit_transform(batch_data)
    
    # Encode data using PCA model    
    batch_data_PCAencoded_df = pd.DataFrame(batch_data_PCAencoded,
                                         index=batch_data.index
                                         )
    
    g = sns.pairplot(batch_data_PCAencoded_df)
    g.fig.suptitle("Batch {}".format(i))
    
"""    
    # Select pairwise PC's to plot
    pc1 = 0
    pc2 = 2
    
    # Encode data using PCA model    
    batch_data_PCAencoded_df = pd.DataFrame(batch_data_PCAencoded[:,[pc1,pc2]],
                                         index=batch_data.index,
                                         columns=['PC {}'.format(pc1), 'PC {}'.format(pc2)])
    
    g = ggplot(aes(x='PC {}'.format(pc1),y='PC {}'.format(pc2)), data=batch_data_PCAencoded_df)  + \
                geom_point(alpha=0.5) + \
                scale_color_brewer(type='qual', palette='Set2') + \
                ggtitle("{} Batches".format(i))
    print(g)
    """

KeyboardInterrupt: 

/home/alexandra/anaconda3/envs/batch_effects/lib/python3.5/site-packages/matplotlib/axis.py:1081: UserWarning: Unable to find pixel distance along axis for interval padding of ticks; assuming no interval padding needed.
  warnings.warn("Unable to find pixel distance along axis "
